In [0]:
%matplotlib notebook

from scipy.io import loadmat
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np

# Load dataset
annots = loadmat('/content/drive/My Drive/VarekaGTNEpochs.mat')
allNonTargetData, allTargetData = annots['allNonTargetData'], annots['allTargetData']

In [0]:
# TODO preprocess data

all_labels = 

In [3]:
# Prepare Keras model
import keras
from keras.layers import (Conv2D, BatchNormalization, Dropout, AveragePooling2D,
Flatten, Dense, Input)
from keras.models import Sequential
from sklearn.model_selection import cross_val_score, ShuffleSplit


def create_model():
  model = Sequential()
  model.add(Conv2D(filters=6, kernel_size=(3, 3),
    activation=keras.activations.elu, input_shape=(3, 1200, 1)))
  model.add(BatchNormalization())
  model.add(Dropout(0.5, seed=0))
  model.add(AveragePooling2D(pool_size=(8, 1)))
  model.add(Flatten())
  model.add(Dense(100, activation=keras.activations.elu))
  model.add(BatchNormalization())
  model.add(Dropout(0.5, seed=0))
  model.add(Dense(2, activation=keras.activations.softmax))
  model.compile(
      optimizer=keras.optimizers.Adam(), loss=keras.losses.BinaryCrossentropy(),
      metrics=['accuracy'])
  return model

seed = 0
es = keras.callbacks.callbacks.EarlyStopping(
    patience=5, verbose=1, restore_best_weights=True)
model = keras.wrappers.scikit_learn.KerasClassifier(
    build_fn=create_model, epochs=30, batch_size=16, verbose=1, callbacks=[es])
split = ShuffleSplit(n_splits=30, train_size=0.75, random_state=seed)
results = cross_val_score(model, cv=split)
print(results.mean())

Using TensorFlow backend.


TypeError: ignored